# Parameter Estimation
First get handle to example model file:

In [1]:
%matplotlib inline
import os,glob

for i in glob.glob('*.cps'):
    kholodenko_model= os.path.abspath(i)
    
print kholodenko_model
print os.path.isfile(kholodenko_model)

/home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/Kholodenko_1.cps
True


Now simulate some synthetic data for sake of running an example:

In [2]:
import PyCoTools,pandas
report= 'parameter_estimation_synthetic_data.txt'
TC=PyCoTools.pycopi.TimeCourse(kholodenko_model,Start=0,End=1000,Intervals=1000,StepSize=1,
                            report_name = report)

## validate that its worked
print pandas.read_csv(TC.kwargs['report_name'],sep='\t').head()

## Give fake data a meaningful name
data1 = TC.kwargs['report_name']

INFO:root:__init__:18:Initializing PyCoTools
INFO:root:__init__:19:Initializing log10ging System
INFO:root:__init__:20:logging config file at: /home/b3053674/anaconda2/lib/python2.7/site-packages/PyCoTools/logging_config.conf
   Time   Mek1-PP     Erk2      Mos   Erk2-P  Erk2-PP    Mos-P     Mek1  \
0     0  10.00000  280.000  90.0000  10.0000  10.0000  10.0000  280.000   
1     1   9.32463  279.821  88.8817  10.1531  10.0262  11.1183  279.883   
2     2   8.66197  279.654  87.7713  10.2949  10.0509  12.2287  279.750   
3     3   8.01289  279.500  86.6685  10.4257  10.0742  13.3315  279.601   
4     4   7.37822  279.358  85.5731  10.5457  10.0959  14.4269  279.437   

    Mek1-P  
0  10.0000  
1  10.7925  
2  11.5882  
3  12.3862  
4  13.1853  


Note that it is completely necessary to have column headings in a data file that exactly match a model variable. Independant variables can be mapped easily by appending the suffic `'_indep'` to the variable. If you had two data sets that stimulated cell cultures at different concentrations for example, you could have `stimulation_indep` as a column heading in both with each containing the amount of stimulant that was used in the experiment. 

# Setup and run single parameter estimation 
Now we're ready to setup and run a parameter estimation

In [3]:
import PyCoTools, os
PE=PyCoTools.pycopi.ParameterEstimation(kholodenko_model,data1)

## default method is GeneticAlgorithm
print PE.kwargs['method']

geneticalgorithm


To change the optimization algorithm use the method kwarg


In [4]:
PE=PyCoTools.pycopi.ParameterEstimation(kholodenko_model,data1,
                                        method='HookJeeves')

AssertionError: HookJeeves is not a copasi PE method. Choose one of: ['CurrentSolutionStatistics', 'DifferentialEvolution', 'EvolutionaryStrategySR', 'EvolutionaryProgram', 'HookeJeeves', 'LevenbergMarquardt', 'NelderMead', 'ParticleSwarm', 'Praxis', 'RandomSearch', 'ScatterSearch', 'SimulatedAnnealing', 'SteepestDescent', 'TruncatedNewton', 'GeneticAlgorithm', 'GeneticAlgorithmSR']

If you spell an algorithm wrong you'll be presented with a list of valid stings for the method parameter. Lets try again

In [5]:
PE=PyCoTools.pycopi.ParameterEstimation(kholodenko_model,data1,
                                        method='HookeJeeves')

## validate (methods are converted into lowercase strings)
PE.kwargs['method']

'hookejeeves'

All algorithm specific parameters should be defined here, along with the instantiation of the ParameterEstimation class

In [6]:
PE=PyCoTools.pycopi.ParameterEstimation(kholodenko_model,data1,
                                        method='HookeJeeves',
                                        iteration_limit = 100,
                                        tolerance = 1e-10)

## check
print 'tolerance = ',PE.kwargs['tolerance']
print 'iteration_limit =', PE.kwargs['iteration_limit']

tolerance =  1e-10
iteration_limit = 100


Now use the `write_item_template` method to create a parameter estimation configuration file:

In [7]:
PE.write_item_template()

'writing template. overwrite_config_file set to false and config_filename is /home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/PEConfigFile.xlsx'

The path to the config file is stored in the `config_filename` kwarg and can be provided on instantiation if desired. Here we use the default:

In [8]:
print PE.kwargs['config_filename']

## check that the config filename exists
print os.path.isfile(PE.kwargs['config_filename'])

/home/b3053674/Documents/PyCoTools/PyCoTools/PyCoToolsTutorial/PEConfigFile.xlsx
True


A parameter estimation configuration file has 7 columns but the last three are not meant to be modified in any way. Each row corresponds to a parameter in your model and by default all kinetic parameters (local and global) and initial concentration parameters are included in the configuration. To exclude a parameter from the estimation simply delete the row corresponding to the parameter you wish to remove. Also, specify the estimation boundaries for each parameter and optionally the starting value. It is common to randomize the start values however and so the start values column is rarely used. If you would like to use the configuration file as to specify start values then you must set the `use_config_start_values` option to `'true'`. 

After modification, save and close the file. In the example, no modifications we're made and so all kinetic and initial concentration parameters are estimated. 

Now use the `set_up()` and `run()` methods. 

In [9]:
PE.set_up()

Open the model and check the parameter estimation task (optionally run the next cell). The ParameterEstimation class has performed the configuration that used to have to do manually. 

In [10]:
import os
os.system('CopasiUI {}'.format(kholodenko_model))
##  remember to close the model before continuing

0

The `run()` method will run the copasi file using `CopasiSE` behind the scenes (provided that an environment variable called `CopasiSE` is set up and pointing to the location of `CopasiSE` on disk. 

In [11]:
PE.run() # will take some time

<PyCoTools.pycopi.run instance at 0x7fcc7e1e5f80>

The `run()` method should take a bit of time and the output will be the parameter estimation data as the algorithm progresses from first to last function evaluation. This means that the best parameters are at the end. Lets take a look

In [12]:
import pandas
import matplotlib.pyplot as plt
import numpy
## lets take a look at the parameter estimation results
pe_data= pandas.read_csv(PE.kwargs['report_name'],sep='\t') 

## top 5 function evaluations
print pe_data.head()

##last 5 function evaluations
print pe_data.tail()

## visualise the best value over function evaluations
rss_string='TaskList[Parameter Estimation].(Problem)Parameter Estimation.Best Value'
plt.figure()
plt.plot(numpy.log10(pe_data.index),numpy.log10(pe_data[rss_string]))
plt.title('RSS versus #FunctionEvaluations')
plt.xlabel('FunctionEvaluations\n(log10)')
plt.ylabel('RSS value\n(log10)')

   [Mek1-PP]_0  [Erk2]_0  [Mos]_0  [Erk2-P]_0  [Erk2-PP]_0  [Mos-P]_0  \
0           10       280       90          10           10         10   
1           10       280       90          10           10         10   
2           10       280       90          10           10         10   
3           10       280       90          10           10         10   
4           10       280       90          10           10         10   

   [Mek1]_0  [Mek1-P]_0  (phosphorylation of MAPKK).KK3  \
0       280          10                         10.9952   
1       280          10                         10.9952   
2       280          10                         10.9952   
3       280          10                         10.9952   
4       280          10                         10.9952   

   (MAPKKK activation).K1  \
0                0.127421   
1                0.127421   
2                0.127421   
3                0.127421   
4                0.127421   

                               

/home/b3053674/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:16: runtimeWarning: divide by zero encountered in log10


## Multiple Data Files
PyCoTools can easily handle multiple data files by giving `ParameterEsimation` a list of data file paths. Lets simulate some more data.

First change the value of a parameter so both sets of simulated data are not identical:

In [13]:
import PyCoTools,os
## Original value of Mek1-PP at time 0 is 10. 
GMQ=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
print GMQ.get_metabolites()['Mek1-PP']

#Use InsertParameters to change it to 20
I=PyCoTools.pycopi.InsertParameters(kholodenko_model,parameter_dict={'Mek1-PP':20})
GMQ=PyCoTools.pycopi.GetModelQuantities(kholodenko_model)
print GMQ.get_metabolites()['Mek1-PP']



{'concentration': 0.00025404200000080036, 'compartment': 'uVol', 'particle_numbers': '1.52987694462e+11'}
{'concentration': 20.0, 'compartment': 'uVol', 'particle_numbers': '1.204428358e+16'}


And Simulate some data:

In [14]:
import PyCoTools,pandas
report= 'parameter_estimation_synthetic_data2.txt'
TC=PyCoTools.pycopi.TimeCourse(kholodenko_model,
                               End=1000,
                               Intervals=1000,
                               StepSize=1,
                               report_name = report)

## validate that its worked
print pandas.read_csv(TC.kwargs['report_name'],sep='\t').head()

## Give fake data a meaningful name
data2 = TC.kwargs['report_name']

   Time   Mek1-PP     Erk2      Mos   Erk2-P  Erk2-PP    Mos-P     Mek1  \
0     0  10.00000  280.000  90.0000  10.0000  10.0000  10.0000  280.000   
1     1   9.32463  279.821  88.8817  10.1531  10.0262  11.1183  279.883   
2     2   8.66197  279.654  87.7713  10.2949  10.0509  12.2287  279.750   
3     3   8.01289  279.500  86.6685  10.4257  10.0742  13.3315  279.601   
4     4   7.37822  279.358  85.5731  10.5457  10.0959  14.4269  279.437   

    Mek1-P  
0  10.0000  
1  10.7925  
2  11.5882  
3  12.3862  
4  13.1853  


Now setup a parameter estimation like before and run. Note that we already have a configuration file with the default filename and therefore do not need to rewrite one. Also, in real life we might consider having a 'Mek1-PP_indep' column in both data files as this is what will discriminate the data. This is not done here for brevity. 

In [15]:
from PyCoTools.pycopi import ParameterEstimation 

report = 'parameter_estimation_data2.txt'
PE=ParameterEstimation(kholodenko_model,[data1,data2],
                      method='GeneticAlgorithm',
                      report_name = report)
PE.set_up()
PE.run()


<PyCoTools.pycopi.run instance at 0x7fcc7dfd9c20>

Lets check the parameter estimation data:

In [63]:
import pandas
print pandas.read_csv(PE.kwargs['report_name'],sep='\t').iloc[-1] #recall that best parameter are at end of file

[Mek1-PP]_0                                                                  165.719000
[Erk2]_0                                                                       0.061016
[Mos]_0                                                                        0.011657
[Erk2-P]_0                                                                   154.178000
[Erk2-PP]_0                                                                   11.024600
[Mos-P]_0                                                                     73.983300
[Mek1]_0                                                                       0.008613
[Mek1-P]_0                                                                     0.000005
(phosphorylation of MAPKK).KK3                                                 0.000002
(MAPKKK activation).K1                                                         0.000011
(dephosphorylation of MAPK-PP).V9                                              0.000001
(phosphorylation of MAPK).k7    

# Visualization 
Use the `plot` keyword to visualize the data

In [ ]:
from PyCoTools.pycopi import ParameterEstimation 

report = 'parameter_estimation_data2.txt'
PE=ParameterEstimation(kholodenko_model,[data1,data2],
                      method='GeneticAlgorithm',
                      report_name = report,
                      plot='true')
PE.set_up()
PE.run()


An alternative is to simply insert the parameters and the open the model using the UI and simulate manually using current solution statistics. In fact this is my preference. 

In [ ]:
import PyCoTools
import os
import pandas

pe_data = pandas.read_csv(report,sep='\t')

print pe_data

InsertParameters(kholodenko_model,parameter_path = report,index = 0)

os.system('CopasiUI {}'.format(kholodenko_model))

